In [1]:
%matplotlib inline
import time
import random
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from numpy import *
import matplotlib as mpl
import matplotlib.cm as cm
from sympy import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import os, sys
from scipy.optimize import OptimizeWarning
import seaborn as sns; import pandas as pd
plt.rcParams.update({'font.size': 14})

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def calcvelocity(rrange,sigma,vdiff,A,vmin):
    sigmaterm = sigma**2
    distances = np.linspace(0,rrange,100)
    I = A*np.exp(-distances**2/sigmaterm)
    return distances, vdiff*np.exp(-I/A)+vmin

def tothecenter(xpos,ypos,angleh):
    value = math.atan(ypos/xpos)*180/math.pi
    if(xpos < 0): angleh = value+180
    else: angleh = value
    if(angleh < 0): angleh += 360
    if(angleh > 360): angleh -= 360
    return angleh, 2

def reflect(xpos,ypos,dirangle):
    value = math.atan(ypos/xpos)*180/np.pi
    angles = np.zeros(3); checks = []
    if(xpos < 0): angles[0] = value+180
    else: angles[0] = value
    if(angles[0] < 0): angles[0] += 360
    if(angles[0] > 360): angles[0] -= 360
    for i in range(1,3): angles[i] = angles[0] + 360 * pow(-1,i)
    for i in range(3): checks.append(abs(angles[i] - dirangle))
    newangle = dirangle + 2 * (angles[checks.index(min(checks))] - dirangle) + 180
    if (newangle > 360): newangle -= 360       
    return newangle, 1, value

def reflectrand(xpos,ypos,angleh):
    value = math.atan(ypos/xpos)*180/math.pi
    if(xpos < 0): angleh = value+180
    else: angleh = value
    angleh += np.random.uniform(-45,45)
    if(angleh < 0): angleh += 360
    if(angleh > 360): angleh -= 360
    return angleh, 1, value
    
def linear(x, m, n): return m*x+n

def disk_line_picking(s,Rin):
    return 4*s/(np.pi*Rin**2)*np.arccos(s/(2*Rin))-2*s**2/(np.pi*Rin**3)*np.sqrt(1-s**2/(4*Rin**2))
    
def calcintensity(laser,laserframe,delay):
    if delay >= 0:   #positive delay - past   - calc velocity due to sensed intensity[i-delay]
        if laserframe < delay: laser[laserframe,j,4] = vfactor*math.exp(-2*laser[0,j,3]/amplitude)+vmin   #if i < delay
        else: laser[laserframe,j,4] = vfactor*math.exp(-2*laser[laserframe-delay,j,3]/amplitude)+vmin
    else:            #negative delay - future
        if laserframe >= fitlength:    #extrapolate intensity value
            popt,pcov = curve_fit(linear, fitwave, laser[laserframe-fitlength:laserframe,j,3],p0 = (0,0))
            futurevalue = linear(-delay,*popt)
            vvalue = vfactor*math.exp(-2*futurevalue/amplitude)+vmin
            if vvalue >= 0 and vvalue <=1: laser[laserframe,j,4] = vvalue
            else:                     # 0 < intensity < 1
                if vvalue > 1: laser[laserframe,j,4] = 1
                if vvalue < 0: laser[laserframe,j,4] = 0 
        else:     # i < delay
            if laserframe < 2: laser[laserframe,j,4] = vfactor*math.exp(-2*laser[laserframe,j,3]/amplitude)+vmin
            else:
                popt,pcov = curve_fit(linear, fitwave[fitlength-laserframe:fitlength], laser[0:laserframe,j,3],p0 = (0,0))
                futurevalue = linear(-delay,*popt)
                if vvalue >= 0 and vvalue <= 1: laser[laserframe,j,4] = vvalue
                else:
                    if vvalue > 1: laser[laserframe,j,4] = 1
                    if vvalue < 0: laser[laserframe,j,4] = 0  
    return laser

def vectorlength(x): return np.sqrt(pow(x[0],2)+pow(x[1],2))

def calcdrifttheory(v,r,tau,delay,base):
    dv = np.gradient(v,base)
    return tau/2*(1-delay/tau)*v*dv+tau*v**2/r

def checkintersection(particle): #collision of particles
    check=True;counter=0
    while check:
        check=False;k=0;m=0
        for k in range(pnumber):
            x1=particle[i,k,0];y1=particle[i,k,1]
            for m in range(pnumber):
                if k<m:
                    x2=particle[i,m,0];y2=particle[i,m,1]
                    distance=math.sqrt((x1-x2)**2+(y1-y2)**2)
                    if distance<diameter*0.99:                      
                        counter+=1
                        intersection=diameter-distance
                        xdist=(particle[i,k,0]-particle[i,m,0])/distance*intersection/2*(1+faster*counter)
                        ydist=(particle[i,k,1]-particle[i,m,1])/distance*intersection/2*(1+faster*counter)
                        particle[i,k,0]+=xdist
                        particle[i,m,0]-=xdist
                        particle[i,k,1]+=ydist
                        particle[i,m,1]-=ydist
                        x1=particle[i,0,0];y1=particle[i,0,1]
                        check=True
        if counter>accuracy: check=False

In [2]:
dt = 0.05
howlong = 50
alldata = np.zeros((howlong,3))
count1=1;count2=0
tauwave = [0.2,0.4,0.65,0.9,1.1]
for i in range(howlong):
    alldata[i,0] = count1
    alldata[i,1] = np.sqrt(240/(tauwave[count2]-dt))
    alldata[i,2] = i
    count1 += 4
    if count1 > 37:
        print(round(alldata[i,1],2))
        count1 = 1
        count2 += 1

40.0
26.19
20.0
16.8
15.12


In [3]:
name = 'test'
np.random.seed(4) 

pnumber = 5
dt = 0.05                                       #iteration time of the camera [s]
stepsexp  = 10000                                    #number of camera pictures
reorientationtimeinput = 1
noise = np.sqrt(240/(reorientationtimeinput-dt))
#noise = 10                                       #Sigma for the Gaussian noise for the angle
delay = 1                                      #Delay, negative value for anticipation
maxvelocityin = 7e-6#5.25e-6                           #velocity of the particle depending on the laser power [m/s]
                                                #Fig 1c; https://www.nature.com/articles/s41467-018-06445-1.pdf
amplitude = 1
vmin = 0                                     #Min velocity
vmax = 1                                        #Max velocity
pxtomum=0.0533e-6                               #px to µm [µm/px]
sigma = 100*pxtomum                                     #Sigma for the Gaussian intensity profile [px]
D_0 = 3.85e-14                                  #measured diffusion coefficient D_0 = kB*T/(6*np.pi*eta*radius)
kB = 1.38064852e-23                              #Boltzmanns constant [J/K]
T = 273.15 + 20                                  #Room temperature [K]
eta = 2.414e-5*10**(247.8/(T-140))               #Viscosity [Pa*s]
offset = 12                                     #offset of the laser [px]
boundaryradius = pxtomum * 300                  #Radius of the boundary [px]
diameter = 2.19e-6                              #diameter of the particle [m]
fitlength = 5                                   #for negative delay, linear fit on the last N measured intensities to extrapolate intensity
diffusion = True                                #Switching diffusion on and off
savename = str(int(noise))+' - '+str(int(delay))
sigmat = 2 * sigma * sigma

accuracy=1000
faster=0.1          #0.2 or 0.1 make it faster, 0 exact
savefolderplus = '.'
shortlength = stepsexp
prefactor = np.sqrt(2*D_0*dt)                 #value for calculation of the random steps
sumhitboundary = 0
fitwave = np.linspace(-fitlength+1,1,fitlength,endpoint=False)
sigmafactor = pow(sigma,2)
vfactor=vmax-vmin
radius = diameter/2
timewave = np.linspace(0,dt*stepsexp,stepsexp,endpoint=False)  #Timewave for sub cam resolution

plothisto = False#True
plotVelocityvsCoM = False

In [4]:
calctime = 0
for p in range(len(alldata)):
#for p in range(25,len(alldata)):
#for p in range(4):
    start_time = time.time() 
    noise = int(alldata[p,1])                             #Sigma for the Gaussian noise for the angle
    delay = int(alldata[p,0])                             #Delay, negative value for anticipation
    name = str(int(noise)) + ' - ' + str(int(delay))
    
    laser = np.zeros((shortlength,pnumber,5))              #laserposition  0 - x; 1 - y; 2 - time at which the laser is set to the new position
                                                 #               3 - Ext Int it measures; 4 - theo speed(Int laser)
    particle = np.zeros((stepsexp,pnumber,8))    #cam detection  0 - x; 1 - y; 2 - time of the average particle position
                                                     #               3 - time at which the the laser is placed 
                                                     #               4 - path number
                                                     #               5 - angle; 6 - out
                                                     #               7 - Distance to center        
    noisesteps = np.zeros((stepsexp,pnumber,2)); anglenoise = np.zeros((stepsexp,pnumber))
    for j in range(pnumber):
        for i in range(2): noisesteps[:,j,i] = prefactor*np.random.normal(0,1,stepsexp) #Create random steps
    if noise != 0: 
        for j in range(pnumber): anglenoise[:,j] = noise*np.random.normal(0,1,stepsexp)           #Create random angle noise
    deltas = np.zeros(2)
    rnoise = np.sqrt(np.random.uniform(0,boundaryradius**2,10*pnumber))
    anglenoise2 = np.random.uniform(0,2*np.pi,10*pnumber)
    anglenoisecos = np.cos(anglenoise2)
    anglenoisesin = np.sin(anglenoise2)
    smaller = pow(diameter,2)
    count = 0
    startx = np.zeros(pnumber); starty = np.zeros(pnumber)
    for j in range(pnumber):
        check = False
        while check == False:
            startx[j] = rnoise[count] * anglenoisecos[count]
            starty[j] = rnoise[count] * anglenoisesin[count]
            if j >= 1:
                pardist = np.zeros(j)
                for k in range(j): pardist[k] = pow(startx[j]-startx[k],2)+pow(starty[j]-starty[k],2)
                if min(pardist) >= smaller: check = True
            else: check = True
            count += 1
    startangle = np.random.uniform(0,360,pnumber)
    particle[0,:,0] = startx; particle[0,:,1] = starty
    particle[0,:,5] = startangle
            
    out = np.zeros(pnumber)
    startpoint = [0]; endpoint = []
    for i in range(1,stepsexp):
        particle[i,:,2] = particle[i-1,:,2] + dt                
        particle[i,:,:2] = particle[i-1,:,:2]                              #Position equals position before
        particle[i,:,5] = particle[i-1,:,5]
        particle[i,:,7] = np.sqrt(pow(particle[i,:,0],2) + pow(particle[i,:,1],2))   #distance to the center
        for j in range(pnumber):
            if particle[i,j,7] > boundaryradius:      #particle is out
                if particle[i,j,7] > 1.1 * boundaryradius: particle[i,j,5], out = tothecenter(particle[i,j,0],particle[i,j,1],particle[i,j,5])  #sendback to the center if something really fails
                else:
                    if out == 0:   #reflect randomly at the boundary
                        particle[i,j,5], out, incidentangle = reflectrand(particle[i,j,0],particle[i,j,1],particle[i,j,5])
                        if j == 0: endpoint.append(i)
            else: 
                if particle[i-1,j,6] != 0 and j == 0: startpoint.append(i)
                out = 0
            particle[i,j,6] = out
            if out == 0: particle[i,j,5] += anglenoise[i,j]   #add noise at the angle if it is inside
            if particle[i,j,5] > 360: particle[i,j,5] -= 360
            if particle[i,j,5] < 0: particle[i,j,5] += 360
            laser[i,j,0] = particle[i,j,0]+offset*pxtomum*np.cos(particle[i,j,5]*np.pi/180)   #laser position      
            laser[i,j,1] = particle[i,j,1]+offset*pxtomum*np.sin(particle[i,j,5]*np.pi/180)
            intensity = 0
            for k in range(pnumber):
                if k != j:
                    xdiff = particle[i-1,j,0]-particle[i-1,k,0]
                    ydiff = particle[i-1,j,1]-particle[i-1,k,1]
                    dist = pow(xdiff,2)+pow(ydiff,2)
                    intensity += 2*np.exp(-dist/sigmat)
            intensity /= pnumber
            if intensity > 1: intensity = 1
            laser[i,j,3] = intensity#amplitude*math.exp(-pow(particle[i,j,7],2)/sigmafactor)        #intensity of the external field
            laser = calcintensity(laser,i,delay)     #calc laser intensity depending on the field intensity 
            if diffusion == True:
                for k in range(2): particle[i,j,k] += noisesteps[i,j,k]         #add diffusion of the particle
            for k in range(2): deltas[k] = particle[i,j,k]-laser[i,j,k]
            particle[i,j,3] = np.sqrt(pow(deltas[0],2)+pow(deltas[1],2))      #calc particle-laser distance
            particle[i,j,4] = maxvelocityin
            for k in range(2): particle[i,j,k] += particle[i,j,4]/particle[i,j,3]*deltas[k]*dt*laser[i,j,4]  #add thermophoretic movement  
            checkintersection(particle)
    if particle[i,0,7] < boundaryradius: endpoint.append(i-1)   
    if startpoint[-1] == shortlength: 
        del(startpoint[-1])
        del([endpoint[-1]])
    #------------------------------------------
    newangle = np.zeros(stepsexp); newangle = particle[:,0,5].copy()    #MSD for the theoretic moving direction
    diff = np.zeros(3)
    for i in range(1,stepsexp):
        diff[0] = abs(newangle[i]-newangle[i-1])
        diff[1] = abs(newangle[i]-newangle[i-1]+360)
        diff[2] = abs(newangle[i]-newangle[i-1]-360)
        mini = min(diff)
        if mini != diff[0]:
            if mini == diff[1]: newangle[i:] += 360
            else: newangle[i:] -= 360
    for i in range(len(endpoint)-1): newangle[endpoint[i]:] -= newangle[endpoint[i]]-newangle[endpoint[i]-1]

    if noise != 0:
        check = True; i = 1
        reorwave= np.zeros((stepsexp,2))
        while check == True:
            diff = np.zeros(stepsexp-i)
            for j in range(i,stepsexp): diff[j-i] = abs(newangle[j] - newangle[j-i])  #Calculate the direction change for i steps
            mean = np.mean(diff[:stepsexp-i])   #Calculate the standard derivation
            reorwave[i,0] = i*dt
            reorwave[i,1] = mean
            if mean/57.3 >= 1: 
                check = False
                reorlen = i
            else: i += 1 
            if i == stepsexp:
                reorlen = i
                i = 666/dt
                check = False       
        reorientationtime = i*dt 
    else: reorientationtime = 666
    
    maxlength = stepsexp
    particles = np.zeros((stepsexp,pnumber,8))
    particles[:,:,0] = particle[:,:,0].copy()
    particles[:,:,1] = particle[:,:,1].copy()
    particles[:,:,2] = particle[:,:,5].copy()
    particles[:,:,0] *= 1e6
    particles[:,:,1] *= 1e6

    directionvector = np.zeros((maxlength,2))
    movingvector = np.zeros((maxlength,2))
    velocity = np.zeros((maxlength,pnumber));stepsize = np.zeros(maxlength)
    velocitywithdiffusion=np.zeros((maxlength,pnumber))
    velocity2 = np.zeros((maxlength,pnumber,2))
    distancetocenter = np.zeros((maxlength,pnumber))

    for j in range(pnumber):
        directionvector[:,0] = np.cos(particles[:,j,2]*np.pi/180)
        directionvector[:,1] = np.sin(particles[:,j,2]*np.pi/180)
        for i in range(1,maxlength):                           #Calc projection of the velocity in the required direction
            distancetocenter[i,j] = np.sqrt(pow(particles[i,j,0],2)+pow(particles[i,j,1],2))
            movingvector[i,0] = particles[i,j,0]-particles[i-1,j,0]
            movingvector[i,1] = particles[i,j,1]-particles[i-1,j,1]
            stepsize[i] = abs(np.dot(directionvector[i,:], movingvector[i,:]))
            velocity[i,j] = stepsize[i]/dt
            velocitywithdiffusion[i,j] = np.sqrt(pow(movingvector[i,0],2)+pow(movingvector[i,1],2))/dt 
            distance = np.sqrt(pow(particles[i,j,0],2)+pow(particles[i,j,1],2))
            if distancetocenter[i,j] < boundaryradius*1e6:
                velocity2[i,j,0] = velocity[i,j]
                velocity2[i,j,1] = velocitywithdiffusion[i,j]
    for j in range(pnumber):
        for i in range(maxlength):
            if velocity2[i,j,0] == 0: velocity2[i,j,:]=np.nan

    string_temp = savefolderplus + '/PlotVelocity/'
    if not os.path.exists(string_temp): os.makedirs(string_temp)
    wave = velocity2[:,:,0]
    binwave=np.linspace(np.amin(wave[~np.isnan(wave)]),min(np.amax(wave[~np.isnan(wave)]),10),20)
    fig=plt.figure(1, figsize = (7,4) )
    plt.subplot(1,1,1)
    for i in range(pnumber):
        wave = velocity2[:,i,0]    
        plt.hist(wave[~np.isnan(wave)], bins=binwave,alpha = 0.2)
        plt.axvline(x=np.nanmean(wave),linewidth=0.5, color = 'k',linestyle='--')
    plt.xlabel('Velocity [$\mu m/s$]')
    plt.ylabel('accurance')
    #plt.text(3,0,'mean step size 1: {:.2f} µm/s'.format(np.mean(velocity)),fontsize=12)
    plt.legend()
    plt.tight_layout() 
    plt.savefig(string_temp + 'Stepsize vs time'+str(name)+'.pdf', format='pdf')
    plt.close(fig)#plt.show() 
    #---------------------------------
    particles[:,:,7] = laser[:,:,3] 
    comlocation = np.zeros((maxlength,5))
    for i in range(maxlength):
        if prod(particles[i,:,0]) != 0:
            comlocation[i,0] = np.nanmean(particles[i,:,0])
            comlocation[i,1] = np.nanmean(particles[i,:,1])
    for i in range(maxlength):
        if comlocation[i,0] == 0:
            pre = i-1
            check = False; checknumber = i+1
            while check == False:
                if checknumber < maxlength:
                    if comlocation[checknumber,0] != 0:
                        post = checknumber
                        check = True
                    else: checknumber += 1
                else: check = True
            comlocation[i,:] = (comlocation[pre,:] + comlocation[post,:])/2

    for i in range(maxlength):
        for j in range(pnumber):
            particles[i,j,6] = np.sqrt(pow(particles[i,j,0]-comlocation[i,0],2)+pow(particles[i,j,1]-comlocation[i,1],2))
        comlocation[i,2] = np.nanmean(particles[i,:,6])
        comlocation[i,3] = np.nanmean(particles[i,:,7])
        comlocation[i,4] = np.nanmean(particles[i,:,2])
    meandisttocom = np.nanmean(comlocation[:,2])
    meanintensity = np.nanmean(comlocation[:,3])    
    test = np.zeros(maxlength)
    for i in range(maxlength):
        count = 0
        for j in range(pnumber):
            if particles[i,j,7] > 0.2: count += 1
        if count >= 3: test[i] = 1
    percentagecluster = sum(test)/maxlength
    #------------------------------------------
    plotvwave = np.zeros((maxlength,pnumber))
    for i in range(maxlength-1): plotvwave[i,:] = laser[i+1,:,4]
    if plotVelocityvsCoM == True:

        string_temp = savefolderplus + '/PlotVelocityvsCoM-intensity/'
        if not os.path.exists(string_temp): os.makedirs(string_temp)
        plotstart = 1
        fig=plt.figure(1, figsize = (14,10) )
        ax = plt.subplot(2,2,1)
        for i in range(pnumber):
            #plt.scatter(particles[plotstart:,i,6],laser[plotstart:,i,4],s=2,c='grey',alpha=0.1)
            plt.scatter(particles[plotstart:,i,6],plotvwave[plotstart:,i],s=2,c='grey',alpha=0.1)
        plt.xlabel('Distance to the CoM [µm]')
        plt.ylabel('intended Velocity')
        plt.xlim([0,np.nanmax(particles[:,:,6])])
        plt.ylim([0,1.05])

        ax = plt.subplot(2,2,2)
        for i in range(pnumber):
            #plt.scatter(particles[plotstart:,i,7],laser[plotstart:,i,4],s=2,c='grey',alpha=0.1)
            plt.scatter(particles[plotstart:,i,7],plotvwave[plotstart:,i],s=2,c='grey',alpha=0.1)
        plt.xlabel('Sensed intensity')
        plt.ylabel('intended Velocity')
        plt.xlim([1,0])
        plt.ylim([0,1.05])

        smoothvalue = 20
        smoothit = np.zeros((maxlength,pnumber+1))
        for i in range(pnumber): smoothit[:,i] = smooth(velocity[:,i],smoothvalue)
        for i in range(maxlength): smoothit[i,-1] = np.mean(smoothit[i,:-1])

        ax = plt.subplot(2,2,3)
        #plt.scatter(comlocation[:,2],comlocation[:,4],s=2,c='grey',alpha=0.1)
        plt.scatter(comlocation[:,2],smoothit[:,-1],s=2,c='grey',alpha=0.1)
        plt.xlabel('Average Distance to the CoM [µm]')
        plt.ylabel('Average velocity [µm/s]')
        plt.xlim([0,np.amax(particles[:,:,6])])

        ax = plt.subplot(2,2,4)
        #plt.scatter(comlocation[:,3],comlocation[:,4],s=2,c='grey',alpha=0.1)
        plt.scatter(comlocation[:,3],smoothit[:,-1],s=2,c='grey',alpha=0.1)
        plt.xlabel('Average sensed Intensity')
        plt.ylabel('Average velocity [µm/s]')
        plt.xlim([1,0])
        plt.tight_layout() 
        plt.savefig(string_temp + 'Velocity vs CoM-intensity'+str(name)+'.pdf', format='pdf')
        plt.savefig(string_temp + 'Velocity vs CoM-intensity'+str(name)+'.png', format='png')
        plt.close(fig)#plt.show() 
    #------------------------------
    connections = int(pnumber*(pnumber-1)/2)
    distances = np.zeros((maxlength,connections))
    calclength = np.zeros(connections)
    boundmum = boundaryradius*1e6# * pxtomum
    trajlength = maxlength*np.ones(pnumber)
    count = 0; totallength = 0
    for j in range(pnumber):
        for k in range(j+1,pnumber):
            calclength[count] = int(min(trajlength[j],trajlength[k]))
            for i in range(int(calclength[count])):
                if distancetocenter[i,j] < boundmum and distancetocenter[i,k] < boundmum:
                    distances[i,count] = np.sqrt(pow(particles[i,j,0]-particles[i,k,0],2)+pow(particles[i,j,1]-particles[i,k,1],2))
            totallength += calclength[count]
            count += 1

    alldistances = np.zeros(int(totallength))
    calcstart = 0; calcend = 0
    for i in range(connections):
        calcend += calclength[i] 
        alldistances[int(calcstart):int(calcend)] = distances[:int(calclength[i]),i]
        calcstart += calclength[i]
    for i in range(int(totallength)):
        if alldistances[i] == 0: alldistances[i] = np.nan
    alldistances = alldistances[~np.isnan(alldistances)] 
    meanexp = np.mean(alldistances)
    meansim = 14.712

    string_temp = savefolderplus + '/PlotParticledistance/'
    if not os.path.exists(string_temp): os.makedirs(string_temp) 
    fig=plt.figure(1, figsize = (7,5) )
    ax = plt.subplot(1,1,1)
    bins = np.linspace(diameter*1e6,2*boundmum,100)
    plt.hist(alldistances,bins = bins,density = 1,color='red',alpha=0.5,label='simulation')
    plt.axvline(x=2*boundmum,linewidth=0.5, color = 'k',linestyle='--')
    theoradius = np.linspace(0,2*boundmum,1000)
    plt.plot(theoradius,disk_line_picking(theoradius,boundmum),color='k',label='theory')
    plt.xlabel('Particle distance [µm]')
    plt.ylabel('Density')
    plt.xlim([diameter*1e6,2*boundaryradius*1e6])
    plt.legend()
    plt.tight_layout() 
    plt.savefig(string_temp+'Particle Distance distribution'+str(name)+'.pdf', format='pdf')
    plt.close(fig)#plt.show() 
    #----------------------------------
    maxvelocity = np.zeros(pnumber)
    for k in range(pnumber):
        value = plotvwave[:,k]
        steps2 = 6
        speedhisto= np.zeros((steps2,4))    #0 - angle, 1 - velocity, 2 - counts, 3 - variance
        speedcollect=np.zeros((steps2,maxlength))
        for i in range(steps2): speedhisto[i,0] = min(value)+(i+1)/steps2*(1-min(value))
        for i in range(maxlength-1):                      #Statistics over the velocity in required direction vs the theoretical velocity
            if value[i] < speedhisto[0,0]:
                speedhisto[0,1] += velocity[i,k]
                speedcollect[0,int(speedhisto[0,2])] = velocity[i,k]
                speedhisto[0,2] += 1
            else:
                for j in range(1,steps2):
                    if value[i] > speedhisto[j-1,0] and value[i] < speedhisto[j,0]:
                        speedhisto[j,1] += velocity[i,k]
                        speedcollect[j,int(speedhisto[j,2])] = velocity[i,k]
                        speedhisto[j,2] += 1
        speedhisto[:,1] /= speedhisto[:,2]
        for i in range(maxlength-1):
            for j in range(steps2):
                if speedcollect[j,i] == 0: speedcollect[j,i] = np.nan
        maxvelocity[k] = float(speedhisto[-1,1])
    #-------------------------------------
    if plothisto == True: 
        x = particles[:,:,0]
        y = particles[:,:,1]
        factor = 1.1
        xmin = factor*x.min()
        xmax = factor*x.max()
        ymin = factor*y.min()
        ymax = factor*y.max()
        gridsizevalue = 15
        string_temp = savefolderplus + '/Plot2DHistogram/'
        if not os.path.exists(string_temp): os.makedirs(string_temp) 
        fig=plt.figure(1, figsize = (8.5,7) ) 
        #https://matplotlib.org/2.0.0/examples/pylab_examples/hexbin_demo.html
        ax = plt.subplot(1,1,1)
        hb = ax.hexbin(x, y, gridsize=gridsizevalue,mincnt=0.01)
        #ax.axis([xmin, xmax, ymin, ymax])
        circle2 = plt.Circle((0, 0), boundaryradius*1e6, color='w', fill=False)
        ax.add_artist(circle2)
        cb = fig.colorbar(hb, ax=ax)
        cb.set_label('counts')
        ax.set_xlabel('x [µm]')
        ax.set_ylabel('y [µm]')
        ax.set_xlim(xmin,xmax)
        ax.set_ylim(ymin,ymax)
        ax.axis('equal')
        plt.tight_layout() 
        plt.savefig(string_temp+'Histogram'+str(name)+'.pdf', format='pdf')
        plt.close(fig)#plt.show() 
        
    #-------------  
    string_temp = savefolderplus + '/DataGeneral/'
    if not os.path.exists(string_temp): os.makedirs(string_temp)           #0  1              2     3    4    5         6    7    8  9    10    
    np.savetxt(string_temp + 'DataGeneral'+str(p)+'.txt', np.transpose([dt,boundaryradius,delay,vmin,vmax,fitlength,sigma,D_0]),fmt='%1.6f')
    string_temp = savefolderplus + '/DataImportant/'                         
    if not os.path.exists(string_temp): os.makedirs(string_temp)          #0       1    2        3              4                5               6            7        8                   
    np.savetxt(string_temp + 'Dataimportant'+str(p)+'.txt', np.transpose([pnumber,delay,noise,reorientationtime,meandisttocom,meanintensity,percentagecluster,meansim,meanexp
    ,maxvelocity[0],maxvelocity[1],maxvelocity[2],maxvelocity[3],maxvelocity[4]]),fmt='%1.6f')

    print("File: ",int(p),"  %s seconds " % round((time.time() - start_time),2),' - noise ',str(noise),' - delay ',str(delay))
    calctime += round((time.time() - start_time),2)
print('Total duration: ',calctime,' s')

File:  0   13.58 seconds   - noise  40  - delay  1
File:  1   12.68 seconds   - noise  40  - delay  5
File:  2   13.09 seconds   - noise  40  - delay  9
File:  3   13.11 seconds   - noise  40  - delay  13
File:  4   13.22 seconds   - noise  40  - delay  17
File:  5   13.11 seconds   - noise  40  - delay  21
File:  6   13.11 seconds   - noise  40  - delay  25
File:  7   12.71 seconds   - noise  40  - delay  29


/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:320: RuntimeWarning: invalid value encountered in true_divide


File:  8   13.27 seconds   - noise  40  - delay  33
File:  9   12.94 seconds   - noise  40  - delay  37
File:  10   13.05 seconds   - noise  26  - delay  1
File:  11   13.2 seconds   - noise  26  - delay  5
File:  12   12.82 seconds   - noise  26  - delay  9
File:  13   13.0 seconds   - noise  26  - delay  13
File:  14   13.14 seconds   - noise  26  - delay  17
File:  15   13.01 seconds   - noise  26  - delay  21
File:  16   12.71 seconds   - noise  26  - delay  25
File:  17   12.91 seconds   - noise  26  - delay  29
File:  18   13.02 seconds   - noise  26  - delay  33
File:  19   13.35 seconds   - noise  26  - delay  37
File:  20   12.73 seconds   - noise  20  - delay  1
File:  21   12.85 seconds   - noise  20  - delay  5
File:  22   12.75 seconds   - noise  20  - delay  9
File:  23   12.78 seconds   - noise  20  - delay  13
File:  24   12.7 seconds   - noise  20  - delay  17
File:  25   13.11 seconds   - noise  20  - delay  21
File:  26   13.06 seconds   - noise  20  - delay  25
File